# Retrain the YOLO model

To retrain the YOLO model we need a prepared dataset of objects images with moderate and severe labels.  We have such a dataset (obtained from RoboFlow) that already contains annotated images, splitted into training and validation datasets.  We will use these training/validation sets to retrain our current YOLO model. A few info on the data structure needed for training:

1. The encode classes of objects we want to teach our model to detect is 0-`moderate` and 1-`severe`.
2. The dataset will be in its own folder, with 2 subfolders in it: `train` and `valid`.  Within each subfolder there will be 2 subfolders: `images` and `labels`.
3. Each image has a corresponding annotation text file in the `labels` subfolder. The annotation text files have the same names as the image files.
4. A dataset descriptor YAML file (data.yaml) points to the datasets and describes the object classes in them. This YAML file is passed to the `train` method of the model to start the training process.

Let's get started!

In [1]:
# If you did not use the Workbench image designed for this Lab, you can uncomment and run the following line to install the required packages.
# !pip install --no-cache-dir --no-dependencies -r requirements.txt

import os
import requests
import zipfile
from tqdm.notebook import tqdm
from ultralytics import YOLO

Next let's load a YOLO model 'yolo8m.pt'

In [2]:
# Load model
model = YOLO('yolov8m.pt')  # load a pretrained model (recommended for training)

## Get the training data

We have provided the following 2 training data sets, available as zip files:  
1) `accident-full.zip`   - to be used to fully re-train the model.
2) `accident-sample.zip` - to be used to partially re-train the model when we don't have the time to fully re-train the model.

During the workshop we will only use the sample dataset.

In [3]:
# Function to retrieve a specific dataset
def retrieve_dataset(dataset_type):

    # Check if the directory exists, if not, create it
    if not os.path.exists("./datasets/"):
        os.makedirs("./datasets/")

    URL = f"https://rhods-public.s3.amazonaws.com/sample-data/accident-data/accident-{dataset_type}.zip"

    # Check if the file exists, if not, download and unzip it
    if not os.path.exists(f"./datasets/accident-{dataset_type}.zip"):
        print("Downloading file...")
        response = requests.get(URL, stream=True)
        total_size = int(response.headers.get('content-length', 0))
        block_size = 1024
        t = tqdm(total=total_size, unit='iB', unit_scale=True)
        with open(f'./datasets/accident-{dataset_type}.zip', 'wb') as f:
            for data in response.iter_content(block_size):
                t.update(len(data))
                f.write(data)
        t.close()
    if os.path.exists(f"./datasets/accident-{dataset_type}.zip"):
        print("Unzipping file...")
        with zipfile.ZipFile(f'./datasets/accident-{dataset_type}.zip', 'r') as zip_ref:
            zip_ref.extractall(path='./datasets/')
    print("Done!")


dataset_type = 'sample'
# dataset_type = 'full' # Use this line instead if you want to retrieve the full dataset
retrieve_dataset(dataset_type)

  0%|          | 0.00/5.44M [00:00<?, ?iB/s]

Unzipping file...
Done!


## Re-training our YOLO model

Let's start by understanding what an 'epoch' is.  Machine learning models are trained with specific datasets passed through an algorithm. Each time the full dataset passes through the algorithm, it is said to have completed an **epoch**. Each **epoch** will further refine the training of the model.

In the training code below we will set various parameters:  
**results = model.train(data='./datasets/accident-sample/data.yaml', epochs=1, imgsz=640, batch=2)**

- epochs: as we want to only demo the process, we will use only 1 epoch.
- imgsz: this is the size of the images the model has to be fed with.
- batch: this is the number of images that the algorithm will process simultaneously. The more the better results, but the more memory it consumes. As we are on constrained resources in this workshop, and this is only a training example, we keep it intentionally low at 2.

In your training run, each **epoch<** will show a summary for both the training and validation phases: lines 1 and 2 show results of the training phase and lines 3 and 4 show the results of the validation phase for each epoch.  

Execute the following cell to start re-training the model!

In [4]:
# Train the model

results = model.train(data='./datasets/accident-sample/data.yaml', epochs=1, imgsz=640, batch=2)

New https://pypi.org/project/ultralytics/8.2.76 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.47 🚀 Python-3.11.7 torch-2.2.2+cpu CPU (Intel Xeon Platinum 8259CL 2.50GHz)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=./datasets/accident-sample/data.yaml, epochs=1, time=None, patience=100, batch=2, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, s

100%|██████████| 755k/755k [00:00<00:00, 75.8MB/s]

Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

  8                  -1  2   3985920  ultralytics.nn.modules.block.C2f             [576, 576, 2, True]           
  9                  -1  1    831168  ultralytics.nn.modules.block.SPPF            [576, 576, 5]                 
 10                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 11             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 12                  -1  2   1993728  ultralytics.nn.modules.block.C2f             [960, 384, 2]                 
 13                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 14             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 15                  -1  2    517632  ultralytics.nn.modules.block.C2f             [576, 192, 2]                 
 16                  -1  1    332160  ultralytics.nn.modules.conv.Conv             [192,

train: Scanning /opt/app-root/src/sentiments/lab-materials/04/datasets/accident-sample/train/labels... 22 images, 0 backgrounds, 0 corrupt: 100%|██████████| 22/22 [00:00<00:00, 1190.67it/s]

...

Plotting labels to runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1         0G      1.156      6.874      1.129          5        640: 100%|██████████| 11/11 [01:08<00:00,  6.26s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:27<00:00,  4.61s/it]


                   all         22         25      0.574      0.214     0.0766     0.0337

1 epochs completed in 0.028 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train/weights/best.pt, 52.0MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.1.47 🚀 Python-3.11.7 torch-2.2.2+cpu CPU (Intel Xeon Platinum 8259CL 2.50GHz)
Model summary (fused): 218 layers, 25840918 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:30<00:00,  5.16s/it]


                   all         22         25      0.575      0.214     0.0776     0.0318
              moderate         22         21       0.15      0.429      0.155     0.0636
                severe         22          4          1          0          0          0
Speed: 3.5ms preprocess, 1186.5ms inference, 0.0ms loss, 200.9ms postprocess per image
Results saved to runs/detect/train


**Note**: after a standard re-training, if we are happy with the results, we could export our model to the ONNX format. 
(you would replace trainX by the traininig session you want to use in the following command)

ObjDetOXModel = YOLO("best.pt").export(format="onnx")

In [5]:
#Export model in onnx format to be deployed in model server
# Load the trained YOLO model
model = YOLO("best.pt")

# Export the model to ONNX format
model.export(format="onnx")

Ultralytics YOLOv8.1.47 🚀 Python-3.11.7 torch-2.2.2+cpu CPU (Intel Xeon Platinum 8259CL 2.50GHz)
YOLOv8-config summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (6.0 MB)

ONNX: starting export with onnx 1.15.0 opset 17...
ONNX: export success ✅ 1.3s, saved as 'best.onnx' (11.7 MB)

Export complete (3.5s)
Results saved to /opt/app-root/src/sentiments/lab-materials/04
Predict:         yolo predict task=detect model=best.onnx imgsz=640  
Validate:        yolo val task=detect model=best.onnx imgsz=640 data=/opt/app-root/src/baggage-images/config.yaml  
Visualize:       https://netron.app


'best.onnx'

## Interpreting our Training Results

A full description of the process and the results for a training against the **full dataset** is available in the Lab instructions.

Now that we have retrained our model let's test it against images with objects suited more broadly for the aviation industry!

**Please open the notebook `04-04-accident-recog.ipynb`**.